1.read data from the sales_samples.csv file and analyse to identify problems


In [0]:
file_schema ="""
id int,
name string,
dop string,
phone long,
amount string,
discount string
"""

In [0]:
sales_raw_df =(
    spark.read.format("csv")
    .option("header","true")
    .schema(file_schema)
    .load("/Volumes/dev/spark_db/datasets/spark_programming/data/sales_sample.csv")
)

In [0]:
new_sales_df =sales_raw_df

In [0]:
sales_raw_df.display()

In [0]:
sales_raw_df.describe().display()

1. convert id from int to string and rename it as transaction_id
2. rename the name col to customer_name
3. convert the dop to date format and rename the col to date_of_purchase
4. rename the phone col to customer_phone

In [0]:
sales_raw_df.withColumnRenamed("name","customer_name").show()

In [0]:
sales_raw_df.withColumnRenamed("amount","purchase_amount")

In [0]:
sales_raw_df = sales_raw_df.selectExpr("*","CAST(id as string) as transaction_id")

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import StringType

sales_raw_df = sales_raw_df.withColumn("id",col("id").cast("string"))

In [0]:
sales_raw_df = sales_raw_df.drop("transaction_id")

In [0]:
sales_raw_df =sales_raw_df.withColumnRenamed("dop","date_of_purchase")

In [0]:
sales_raw_df.display()

In [0]:
from pyspark.sql.functions import try_to_date

sales_raw_df = sales_raw_df.withColumn("date_of_purchase",try_to_date(col("date_of_purchase"),"MM/dd/yyyy"))

In [0]:
sales_raw_df.show()

In [0]:
new_sales_df.display()

excerice to cast , datatypes, date

In [0]:
sales_df = new_sales_df.selectExpr(
    "cast(id as string) as transaction_id",
    "name as customer_name",
    "nvl(try_cast(dop as date),to_date(dop,'dd-MM-yyyy')) as date_of_purchase",
    "cast(phone as string) as customer_phone",
    "cast(amount as long) as purchase_amount",
    "nvl(try_cast(discount as double),0) as applied_discount",

).filter("purchase_amount is not null and purchase_amount < 200000")
sales_df.display()